In [3]:
# set the right profile
%env AWS_PROFILE=staging

env: AWS_PROFILE=staging


In [4]:
import boto3
from aws_requests_auth.boto_utils import BotoAWSRequestsAuth
from elasticsearch import Elasticsearch, RequestsHttpConnection

# staging
es_host = "search-quilt-t-search-r1um6raayix1-uuvm5mzf5jgdke7hs7wskz3cre.us-east-1.es.amazonaws.com"

auth = BotoAWSRequestsAuth(
    aws_host=es_host,
    aws_region='us-east-1',
    aws_service='es'
)

elastic = Elasticsearch(
    hosts=[{"host": f"{es_host}", "port": 443}],
    http_auth=auth,
    use_ssl=True,
    verify_certs=True,
    connection_class=RequestsHttpConnection,
    timeout=27
)

In [5]:
IMG_EXTS = [
    '.jpg',
    '.jpeg',
    '.png',
    '.gif',
    '.webp',
    '.bmp',
    '.tiff',
    '.tif',
]

SAMPLE_EXTS = [
]

README_KEYS = ['README.md', 'README.txt', 'README.ipynb']

SUMMARIZE_KEY = 'quilt_summarize.json'

In [67]:
sample_body = {
    'query': {
        'bool': {
            'must': [{'terms': {'ext': SAMPLE_EXTS}}],
            'must_not': [
                {'terms': {'key': README_KEYS + [SUMMARIZE_KEY]}},
                {'wildcard': {'key': '*/' + SUMMARIZE_KEY}},
            ],
        },
    },
    'collapse': {
        'field': 'key',
        'inner_hits': {
            'name': 'latest',
            'size': 1,
            'sort': [{'last_modified': 'desc'}],
            '_source': ['key', 'version_id'],
        },
    },
}

img_body = {
    'query': {'terms': {'ext': IMG_EXTS}},
    'collapse': {
        'field': 'key',
        'inner_hits': {
            'name': 'latest',
            'size': 1,
            'sort': [{'last_modified': 'desc'}],
            '_source': ['key', 'version_id'],
        },
    },
}


img_regex_body = {
    "query": {
        "regexp": {
            "ext":  '.*\.(csv|ipynb|json|md|parquet|pdf|rmd|tsv|txt|vcf|xls|xlsx)(.gz)?',
        }
    }
}

xbody = {
    'query': {
        'bool': {
            'must': [{'regexp': {'ext': r'.*\.(csv|ipynb|json|md|parquet|pdf|rmd|tsv|txt|vcf|xls|xlsx)(.gz)?'}}],
            'must_not': [
                {'terms': {'key': README_KEYS + [SUMMARIZE_KEY]}},
                {'wildcard': {'key': '*/' + SUMMARIZE_KEY}},
            ],
        },
    },
    'collapse': {
        'field': 'key',
        'inner_hits': {
            'name': 'latest',
            'size': 1,
            'sort': [{'last_modified': 'desc'}],
            '_source': ['key', 'version_id'],
        },
    },
}

simple = {
    "query": {
        "simple_query_string" : {
            "query": "akarve.*\.png"
            "analyze_wildcard": False,
            "fields": ['key'],
        }
    }
}

SyntaxError: invalid syntax (<ipython-input-67-bc293fd6d013>, line 69)

In [79]:
to_search = "searchminimal"
_source = ['key']

elastic.search(
    index=to_search,
    body=simple,
    _source=_source,
    size=100,
    terminate_after="10"
)

{'took': 174,
 'timed_out': False,
 'terminated_early': True,
 '_shards': {'total': 10, 'successful': 10, 'skipped': 0, 'failed': 0},
 'hits': {'total': 100,
  'max_score': 1.0,
  'hits': [{'_index': 'searchminimal-reindex-vea01e39',
    '_type': '_doc',
    '_id': 'akarve/fcs/Beckman Coulter - MoFlo XDP.fcs:null',
    '_score': 1.0,
    '_source': {'key': 'akarve/fcs/Beckman Coulter - MoFlo XDP.fcs'}},
   {'_index': 'searchminimal-reindex-vea01e39',
    '_type': '_doc',
    '_id': 'akarve/fcs/Cytek DxP10 - PE COMP BEADS009.fcs:null',
    '_score': 1.0,
    '_source': {'key': 'akarve/fcs/Cytek DxP10 - PE COMP BEADS009.fcs'}},
   {'_index': 'searchminimal-reindex-vea01e39',
    '_type': '_doc',
    '_id': 'akarve/fcs/attachments/Extracted.FCS.headers.zip:null',
    '_score': 1.0,
    '_source': {'key': 'akarve/fcs/attachments/Extracted.FCS.headers.zip'}},
   {'_index': 'searchminimal-reindex-vea01e39',
    '_type': '_doc',
    '_id': 'akarve/fcs/attachments/Verity_Software_House_FCS_Key